Author: Cezary Januszek

*Created on Tuesday, November 7th 2023*
# Flight delays prediction

#### Research question: *What is the influence of the different airlines/airports on the flight delays?*

The goal of this challenge is to investigate US flight delays and cancellations data from 2015.

First some exploratory data analysis is done to try to find some insights from general statistics about the airlines, airports and average flight delays. Second, using this same data the goal consists in creating a flight delay prediction model by implementing a:
1) baseline model 
2) complex model (Neural Network or other Deep Learning method)

The second model should have better performance than the baseline due to its increased complexity, which will make it also more difficult to explain its predictions. This is known as the accuracy-interpretability trade-off. Therefore, an interpretability framework will be added to the second model to allow to better understand and potentially explain the obtained predictions.

In [3]:
# Imports
%load_ext autoreload
%autoreload 2

from IPython.core.pylabtools import figsize
figsize(10, 8)

import os
import pandas as pd
import numpy as np
import datetime

from src.helper_functions import convert_to_time, get_arrival_date


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# I. Data cleaning and pre-processing notebook

Our data consists of 3 dataframes:
1. **Airlines** - associates IATA CODE for the airline with its airline name
2. **Airports** - contains information about airports: IATA code, airport name, city, state, latitude and longitude
3. **Flights** - contains all information about flights with the following essential variables:

    - *YEAR, MONTH, DAY, DATE_OF_WEEK*: date of flight
    - *AIRLINE*: IATA CODE for the airline
    - *FLIGHT_NUMBER*: id number of the flight
    - *TAIL_NUMBER*: code to identify a specific plane
    - *ORIGIN_AIRPORT and DESTINATION_AIRPORT*: origin and destination airport of the flight
    - *SCHEDULED_DEPARTURE and SCHEDULED_ARRIVAL*: scheduled times of flight departure and landing
    - *DEPARTURE_TIME and ARRIVAL_TIME*: real times of flight takeoff and landing
    - *DEPARTURE_DELAY and ARRIVAL_DELAY*: differences between scheduled and real times of flight departure and arrival
    - *DISTANCE*: travel distance between origin and destination airports (in miles)
    - *SCHEDULED_TIME*: scheduled travel time between origin and destination airports
    

In [24]:
#---------------------------
# DATA LOADING
#---------------------------

#Load data from csv files
airlines_df = pd.read_csv(os.path.abspath('./Data/csv/airlines.csv'))
airports_df = pd.read_csv(os.path.abspath('./Data/csv/airports.csv'))
flights_df = pd.read_csv(os.path.abspath('./Data/csv/flights.csv'), low_memory=False)


# Set IATA code as index for the airlines and airports df for easier use
airlines_df = airlines_df.set_index('IATA_CODE')
airports_df = airports_df.set_index('IATA_CODE')

## Data cleaning

The very first step of our data science pipeline consists in cleaning the dataset to make sure that only useful information for the next steps is kept and that outliers or missing values won't influence the performance of the prediction models.

### 1) Dataframes inspection and checking missing values and duplicates

In [12]:
# AIRLINES - everything clear here
airlines_df

,AIRLINE
IATA_CODE,
UA,United Air Lines Inc.
AA,American Airlines Inc.
US,US Airways Inc.
F9,Frontier Airlines Inc.
B6,JetBlue Airways
OO,Skywest Airlines Inc.
AS,Alaska Airlines Inc.
NK,Spirit Air Lines
WN,Southwest Airlines Co.


In [13]:
# AIRPORTS - checking missing values
print(airports_df.isnull().sum())
airports_df[(airports_df['LATITUDE'].isnull()) | (airports_df['LONGITUDE'].isnull())]

AIRPORT      0
CITY         0
STATE        0
COUNTRY      0
LATITUDE     3
LONGITUDE    3
dtype: int64


,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
IATA_CODE,,,,,,
ECP,Northwest Florida Beaches International Airport,Panama City,FL,USA,NaN,NaN
PBG,Plattsburgh International Airport,Plattsburgh,NY,USA,NaN,NaN
UST,Northeast Florida Regional Airport (St. August...,St. Augustine,FL,USA,NaN,NaN


In [14]:
# Manual insertion of missing latitude and longitude values for the 3 airports
airports_df.loc['ECP', ['LATITUDE', 'LONGITUDE']] = [30.3417, -85.7973]
airports_df.loc['PBG', ['LATITUDE', 'LONGITUDE']] = [44.6504, -73.4674]
airports_df.loc['UST', ['LATITUDE', 'LONGITUDE']] = [29.9554, -81.3371]
# check
print(airports_df.isnull().sum())

AIRPORT      0
CITY         0
STATE        0
COUNTRY      0
LATITUDE     0
LONGITUDE    0
dtype: int64


In [15]:
# FLIGHTS
#first inspection of flights df columns
flights_df.columns

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'],
      dtype='object')

In [16]:
# check of missing values percentage per column
pd.DataFrame(round(flights_df.isnull().sum(axis=0)/len(flights_df)*100, 3), columns=['Missing values percentage'])

,Missing values percentage
YEAR,0.000
MONTH,0.000
DAY,0.000
DAY_OF_WEEK,0.000
AIRLINE,0.000
FLIGHT_NUMBER,0.000
TAIL_NUMBER,0.253
ORIGIN_AIRPORT,0.000
DESTINATION_AIRPORT,0.000
SCHEDULED_DEPARTURE,0.000


In [17]:
# check of the percentage of diverted and cancelled flights
pd.DataFrame(round(flights_df[['DIVERTED', 'CANCELLED']].sum(axis=0)/len(flights_df)*100, 3), columns=['Percentage of flights'])

,Percentage of flights
DIVERTED,0.261
CANCELLED,1.545


Several columns have high missing values percentage: *CANCELLATION_REASON* and *DELAY reasons*. As they should provide information about why a cancellation or delay occurred, we would not use them in any case for our prediction model, so they can be dropped without having any issues later on. 

Additionally, we see that diverted and cancelled flights are a very small percentage of all the flights in our dataset and do not actually provide direct information about delays. Therefore, these fligths are also dropped from the data. One could argue that diverted flights are actually large delays, however within our dataset we are not able to estimate the time it takes for passengers to be transported to the destination from the airport that the plane was diverted to. The same goes for cancellations, those could be defined as infinite delay, but it would not help us to train an efficient model for delay prediction. 

In [18]:
# Dropping columns with high missing values factor and DIVERTED and CANCELLED columns
flights_df = flights_df.drop(['CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY', 
                              'DIVERTED', 'CANCELLED'], axis=1)

# check if there are not different data entries for the same flight
flights_df[flights_df[['YEAR', 'MONTH', 'DAY', 'AIRLINE', 'FLIGHT_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']].duplicated()]

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY


**IMPORTANT:** Here, we have to define what kind of delays we will be considering for our prediction model - departure delays and/or arrival delays. 

For the scope of this assignment, we **focus strictly on ARRIVAL DELAYS**, as those are the most interesting one for travelers. Arrival delays can negetively affect flight transfers at destination airport or travelers' plans once they arrive at their destination. On the other hand, departure delays strongly influence corresponding arrival delays, but otherwise rarely make travelers change their plans as they are known in very short advance on their own (just before departure or once passengers are already on board). Very often departure delays are provoked by previous flight arrival delay, which confirms that predicting the latter seems more impactful.

In [19]:
# Dropping other columns not used for the task of arrival delays prediction 
flights_df = flights_df.drop(['WHEELS_OFF', 'WHEELS_ON', 'AIR_TIME', 'ELAPSED_TIME', 'FLIGHT_NUMBER', 'TAIL_NUMBER'], axis=1)

# for the resting columns only a small percentage is NaN so we can drop these rows even if we lose some data
flights_df = flights_df.dropna()
# drop duplicates if there are any
flights_df = flights_df.drop_duplicates()

## Data pre-processing

In [20]:
#----------------------------------
# DATES and TIMES
#----------------------------------

# Create DATES column from YEAR, MONTH, DAY
flights_df['DATE'] = pd.to_datetime(flights_df[['YEAR', 'MONTH', 'DAY']])

# Convert departure and arrival times to HH:MM format
# initially the departure and arrival times are in either int or float format
flights_df['SCHEDULED_DEPARTURE'] = flights_df['SCHEDULED_DEPARTURE'].apply(convert_to_time)
flights_df['SCHEDULED_ARRIVAL'] = flights_df['SCHEDULED_ARRIVAL'].apply(convert_to_time)
flights_df['DEPARTURE_TIME'] = flights_df['DEPARTURE_TIME'].apply(convert_to_time)
flights_df['ARRIVAL_TIME'] = flights_df['ARRIVAL_TIME'].apply(convert_to_time)

# Get arrival date based on departure and arrival times
flights_df['ARRIVAL_DATE'] = flights_df[['SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL', 'DATE']].apply(get_arrival_date, axis=1)

In [21]:
#-------------------------------------------
# Airline names
#-------------------------------------------

# Dictionary for airline names with IATA code as key and airline name as value
airline_name = airlines_df['AIRLINE'].to_dict()

# Replace IATA code by full name for each airline for readability reason
# Airports codes are not replaced as in this case IATA codes are often generally used and more easily interpreted
flights_df['AIRLINE'] = flights_df['AIRLINE'].map(airline_name)

In [22]:
# Final dropping of columns 
flights_df = flights_df.drop(['YEAR', 'MONTH', 'DAY'], axis=1).reset_index(drop=True)
flights_df

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,SCHEDULED_TIME,DISTANCE,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DATE,ARRIVAL_DATE
0,4,Alaska Airlines Inc.,ANC,SEA,00:05:00,23:54:00,-11.0,21.0,205.0,1448,4.0,04:30:00,04:08:00,-22.0,2015-01-01,2015-01-01
1,4,American Airlines Inc.,LAX,PBI,00:10:00,00:02:00,-8.0,12.0,280.0,2330,4.0,07:50:00,07:41:00,-9.0,2015-01-01,2015-01-01
2,4,US Airways Inc.,SFO,CLT,00:20:00,00:18:00,-2.0,16.0,286.0,2296,11.0,08:06:00,08:11:00,5.0,2015-01-01,2015-01-01
3,4,American Airlines Inc.,LAX,MIA,00:20:00,00:15:00,-5.0,15.0,285.0,2342,8.0,08:05:00,07:56:00,-9.0,2015-01-01,2015-01-01
4,4,Alaska Airlines Inc.,SEA,ANC,00:25:00,00:24:00,-1.0,11.0,235.0,1448,5.0,03:20:00,02:59:00,-21.0,2015-01-01,2015-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5714003,4,JetBlue Airways,LAX,BOS,23:59:00,23:55:00,-4.0,22.0,320.0,2611,4.0,08:19:00,07:53:00,-26.0,2015-12-31,2016-01-01
5714004,4,JetBlue Airways,JFK,PSE,23:59:00,23:55:00,-4.0,17.0,227.0,1617,3.0,04:46:00,04:30:00,-16.0,2015-12-31,2016-01-01
5714005,4,JetBlue Airways,JFK,SJU,23:59:00,23:50:00,-9.0,17.0,221.0,1598,8.0,04:40:00,04:32:00,-8.0,2015-12-31,2016-01-01
5714006,4,JetBlue Airways,MCO,SJU,23:59:00,23:53:00,-6.0,10.0,161.0,1189,3.0,03:40:00,03:30:00,-10.0,2015-12-31,2016-01-01


In [23]:
#--------------------------------
# ORIGIN and DESTINATION airports
#--------------------------------
# For the month of October instead of having IATA codes there are some number IDs in the ORIGIN_AIRPORT and DESTINATION_AIRPORT columns
flights_df[flights_df['DATE'].apply(lambda x: x.month) == 10].head()


,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,SCHEDULED_TIME,DISTANCE,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DATE,ARRIVAL_DATE
4299046,4,American Airlines Inc.,14747,11298,00:05:00,00:15:00,10.0,15.0,237.0,1660,4.0,06:02:00,05:43:00,-19.0,2015-10-01,2015-10-01
4299047,4,Delta Air Lines Inc.,14771,13487,00:05:00,00:16:00,11.0,14.0,213.0,1589,7.0,05:38:00,05:28:00,-10.0,2015-10-01,2015-10-01
4299048,4,Spirit Air Lines,12889,13487,00:05:00,00:00:00,-5.0,15.0,177.0,1299,4.0,05:02:00,04:48:00,-14.0,2015-10-01,2015-10-01
4299049,4,American Airlines Inc.,12892,13303,00:10:00,00:07:00,-3.0,28.0,296.0,2342,7.0,08:06:00,08:13:00,7.0,2015-10-01,2015-10-01
4299050,4,American Airlines Inc.,14771,11057,00:10:00,00:08:00,-2.0,12.0,291.0,2296,13.0,08:01:00,07:50:00,-11.0,2015-10-01,2015-10-01


In [17]:
# Completing the ORIGIN and DESTINATION airport columns by replacing the airport ID for the month of october by assocaited IATA code
# Data was taken from https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FGJ&QO_fu146_anzr=b0-gvzr (Bureau of Transportation Statistics)
id_airports = pd.read_csv(os.path.abspath('./Data/csv/AIRPORT_ID_ORIGIN.csv'))
code_airports= pd.read_csv(os.path.abspath('./Data/csv/AIRPORT.csv'))
# merge ID numbers with IATA code by airport description
airport_code_id = id_airports.merge(code_airports, on='Description')
airport_code_id = airport_code_id.rename(columns={'Code_x': 'Code', 'Code_y': 'IATA_CODE'})
airport_code_id['AIRPORT'] = airport_code_id['Description'].apply(lambda x: x.split(':')[-1])
airport_code_id['CITY'] = airport_code_id['Description'].apply(lambda x: x.split(',')[0])
airport_code_id['STATE'] = airport_code_id['Description'].apply(lambda x: x.split(',')[-1].split(':')[0])
airport_code_id['Code'] = airport_code_id['Code'].apply(str)
# create ID number to IATA dictionary
id_airports = airport_code_id.set_index('Code')['IATA_CODE'].to_dict()
id_airports

{'10001': '01A',
 '10003': '03A',
 '10004': '04A',
 '10005': '05A',
 '10006': '06A',
 '10007': '07A',
 '10008': '08A',
 '10009': '09A',
 '10010': '1B1',
 '10011': 'DQR',
 '11398': 'DQR',
 '10012': '1N7',
 '10013': '8F3',
 '10014': 'A01',
 '10015': 'A02',
 '10016': 'A03',
 '10017': 'A04',
 '10018': 'A05',
 '10019': 'A06',
 '10020': 'A07',
 '10021': 'A08',
 '10022': 'A09',
 '10023': 'TLK',
 '10024': 'A11',
 '10025': 'A12',
 '10026': 'A13',
 '10027': 'A14',
 '10028': 'A15',
 '10029': 'A16',
 '10030': 'A17',
 '10031': 'A18',
 '10033': 'DQH',
 '11388': 'DQH',
 '10034': 'A21',
 '10035': 'A22',
 '10036': 'A23',
 '10037': 'A24',
 '10038': 'A25',
 '10039': 'A26',
 '10040': 'A27',
 '10041': 'A28',
 '10042': 'A29',
 '10043': 'A30',
 '10044': 'A31',
 '10047': 'A34',
 '10049': 'A36',
 '10050': 'A37',
 '10051': 'A38',
 '10052': 'A39',
 '10053': 'A40',
 '10054': 'A41',
 '10055': 'A42',
 '10056': 'A43',
 '10058': 'A45',
 '10059': 'A46',
 '10060': 'A47',
 '10061': 'A48',
 '10062': 'A49',
 '10063': 'A50

In [18]:
# replace ID numbers with corresponding IATA codes
flights_df['ORIGIN_AIRPORT'] = flights_df['ORIGIN_AIRPORT'].replace(id_airports)
flights_df['DESTINATION_AIRPORT'] = flights_df['DESTINATION_AIRPORT'].replace(id_airports)

# check that all ID numbers were replace with corresponding IATA codes
flights_df[flights_df['DATE'].apply(lambda x: x.month) == 10]['ORIGIN_AIRPORT'].unique()

array(['SEA', 'SFO', 'LAS', 'LAX', 'SLC', 'ANC', 'DEN', 'PHX', 'FAI',
       'BQN', 'PSE', 'ABR', 'BIS', 'GRK', 'HNL', 'ORD', 'SAT', 'IAH',
       'EWR', 'BOS', 'MSP', 'GEG', 'XNA', 'OKC', 'COU', 'BNA', 'HOU',
       'SGF', 'STL', 'RDD', 'BSM', 'TUL', 'GNV', 'ONT', 'TUS', 'FSM',
       'DCA', 'PHL', 'MCO', 'TPA', 'FAR', 'SJT', 'ACT', 'FSD', 'DFW',
       'CLT', 'ORF', 'BWI', 'PIT', 'CLL', 'CHO', 'BJI', 'MAF', 'PIA',
       'MCI', 'DVL', 'HRL', 'MSY', 'PDX', 'VPS', 'BRO', 'TYR', 'LAW',
       'CHS', 'MFE', 'BUF', 'RDU', 'JAX', 'LWS', 'BTR', 'SPS', 'ICT',
       'SAV', 'GSO', 'MSN', 'GRB', 'ALB', 'OAK', 'MDW', 'MHT', 'RNO',
       'SMF', 'RIC', 'HSV', 'BOI', 'AGS', 'ABI', 'EUG', 'SBA', 'EAU',
       'DIK', 'MFR', 'JFK', 'SHV', 'DHN', 'CMH', 'LBB', 'ATL', 'PVD',
       'FNT', 'CRP', 'AEX', 'MGM', 'CWA', 'MKE', 'DTW', 'FLL', 'LGA',
       'BHM', 'HLN', 'SPI', 'BIL', 'CLE', 'SYR', 'ATW', 'TLH', 'DSM',
       'LCH', 'IAD', 'IND', 'ABQ', 'OMA', 'PBI', 'MEM', 'DAY', 'ELP',
       'ROC', 'CAE',

In [31]:
# save the resulting cleaned flights data to pickle file
flights_df.to_pickle(os.path.abspath('./Data/flights.pkl'))

Last, but not least we check that all airports have complete details.

In [147]:
airports_df = pd.read_csv('./Data/csv/airports.csv')
flights_df = pd.read_pickle(os.path.abspath('./Data/flights.pkl'))

In [152]:
# Check that for all IATA codes airports present in flights data there are corresponding details in airport df

# check if the lists of unique IATA codes in origin and destination airport are the same
iata_diff = len(set(flights_df['ORIGIN_AIRPORT'].unique()).difference(set(flights_df['DESTINATION_AIRPORT'].unique())))
print('Both sets are the same: ', iata_diff == 0)

# find missing IATA code (if any) in the airport df and complete its details
missing_elem = pd.Series(flights_df['ORIGIN_AIRPORT'].unique())[~pd.Series(flights_df['ORIGIN_AIRPORT'].unique()).isin(airports_df.index)]
# complete airport df
airports_df = pd.concat([airports_df, airport_code_id[airport_code_id['IATA_CODE'] == missing_elem.values[0]][['IATA_CODE', 'AIRPORT', 'CITY', 'STATE']].set_index('IATA_CODE')])
airports_df.loc['BSM', 'COUNTRY'] = 'USA'
airports_df.loc['BSM', ['LATITUDE', 'LONGITUDE']] = [30.18999, -97.66866]

airports_df

Both sets are the same:  True


,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
IATA_CODE,,,,,,
ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
...,...,...,...,...,...,...
WYS,Westerly State Airport,West Yellowstone,MT,USA,44.68840,-111.11764
XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,USA,36.28187,-94.30681
YAK,Yakutat Airport,Yakutat,AK,USA,59.50336,-139.66023


In [ ]:
# save airports
airports_df.to_pickle(os.path.abspath('./Data/airports.pkl'))